In [ ]:
%%bash

# Define target JSON
target="PDL1.json"
target_path="/workspace/settings_target/$target"
log_file="/workspace/outputs/bindcraft_log.txt"

echo "[INFO] Checking target settings file..."
if [ ! -f "$target_path" ]; then
  echo "[ERROR] Target file '$target' not found in /workspace/settings_target/"
  exit 1
fi

echo "[INFO] Starting BindCraft with target '$target'"
echo "[INFO] Logging output to $log_file"

nohup python /app/bindcraft/bindcraft.py \
  --settings "$target_path" \
  --filters /workspace/settings_filters/default_filters.json \
  --advanced /workspace/settings_advanced/default_4stage_multimer.json \
  > "$log_file" 2>&1 &

echo "[INFO] BindCraft is now running in the background (PID=$!)"


In [ ]:
!tail -n 50 -f /workspace/outputs/bindcraft_log.txt

In [ ]:
import ipywidgets as widgets
from IPython.display import display
uploader = widgets.FileUpload(accept='.json', multiple=False)
display(uploader)